## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [1]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [2]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [3]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [4]:
sp500.head()

,permno,start,ending,date,ret
0,15202,1999-07-01,2023-12-29,2000-01-31,0.059468
1,19692,1957-03-01,2000-11-16,2000-01-31,-0.314607
2,39490,1997-07-28,2023-12-29,2000-01-31,-0.011844
3,20626,1954-04-20,2017-08-31,2000-01-31,-0.128157
4,10147,1996-03-28,2016-09-07,2000-01-31,-0.025172


### Step 3: Add Other Company Identifiers from CRSP.MSENAMES
- You don't need this step if only PERMNO is required
- This step aims to add TICKER, SHRCD, EXCHCD and etc. 

In [5]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [6]:
mse.sample(5)

,comnam,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker
40314,FLAKEY JAKES INC,33890310,1983-10-11,1986-05-13,36741,10,3,5812,FJAK
78039,STERLING HEALTHCARE GROUP INC,85932610,1994-06-01,1995-08-03,80696,11,2,8090,DRZ
9058,POWERSHARES E T F TRUST II,73937B77,2011-05-05,2018-06-03,12753,73,4,6726,SPLV
43748,HYBRIDOMA SCIENCES INC,44860510,1986-04-01,1986-05-29,42981,11,3,2830,HYBD
70305,MIDLAND BANCORP INC,59742110,1982-11-01,1984-04-24,77965,10,3,6711,MDBC


In [7]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
2091,23309,1970-04-23,2007-10-24,2000-01-31,-0.117647,HILTON HOTELS CORP,43284810,1968-01-02,2002-01-01,11,1,7011,HLT
260520,53065,1992-10-30,2023-12-29,2005-09-30,-0.040396,INTERPUBLIC GROUP COS INC,46069010,2004-06-10,2023-12-29,11,1,7311,IPG
566018,55976,1982-08-19,2023-12-29,2012-05-31,0.124045,WAL MART STORES INC,93114210,2012-03-01,2014-01-06,11,1,5311,WMT
721362,75577,2000-04-18,2015-12-28,2015-10-30,0.049321,ALTERA CORP,02144110,2004-06-10,2015-12-24,11,3,3670,ALTR
141338,67467,1998-01-20,2013-02-13,2003-02-28,-0.116000,BIG LOTS INC,08930210,2002-01-02,2004-06-09,11,1,5331,BLI


### Step 4: Add Compustat Identifiers
- Link with Compustat's GVKEY and IID if need to work with fundamental data
- Linkage is done through crsp.ccmxpf_linktable

In [8]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [9]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]
sp500ccm.sample(5)

,permno,start,ending,date,ret,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker,gvkey,iid,linktype,linkprim,linkdt,linkenddt
41289,88661,2004-09-01,2023-12-29,2005-02-28,-0.010160,COACH INC,18975410,2004-06-10,2006-06-29,11,1,3199,COH,140541,01,LC,P,2000-10-05,2024-03-05 21:35:52.295663
129809,86158,2006-11-17,2023-12-29,2016-11-30,0.072639,COGNIZANT TECHNOLOGY SOLS CORP,19244610,2004-06-10,2023-12-29,11,3,7370,CTSH,111864,01,LU,P,1998-06-19,2024-03-05 21:35:52.295663
169836,10107,1994-06-07,2023-12-29,2022-04-29,-0.099867,MICROSOFT CORP,59491810,2004-06-10,2023-12-29,11,3,7370,MSFT,012141,01,LC,P,1986-03-13,2024-03-05 21:35:52.295663
146979,25081,1995-12-01,2023-12-29,2019-02-28,0.106299,COMERICA INC,20034010,2004-06-10,2023-12-29,11,1,6021,CMA,003231,01,LC,P,1972-12-14,2024-03-05 21:35:52.295663
23088,57665,1988-11-03,2023-12-29,2002-10-31,0.092867,NIKE INC,65410610,2002-01-02,2004-06-09,11,1,3021,NKE,007906,01,LU,P,1980-12-02,2024-03-05 21:35:52.295663


In [10]:
# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]
sp500ccm.sample(5)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
63624,2008-01-31,14008,AMGEN INC,03116210,11,3,2830,AMGN,001602,01,1992-01-02,2023-12-29,0.003230
22988,2002-10-31,59379,MELLON FINANCIAL CORP,58551A10,11,1,6021,MEL,007238,01,1976-07-01,2007-06-29,0.096028
78286,2009-12-31,63765,SOUTHWESTERN ENERGY CO,84546710,11,1,1311,SWN,009904,01,2008-06-06,2017-04-03,0.096451
156985,2020-07-31,90319,ALPHABET INC,02079K30,11,3,7375,GOOGL,160329,01,2006-04-03,2023-12-29,0.049293
148704,2019-05-31,89070,ZIMMER BIOMET HOLDINGS INC,98956P10,11,1,3842,ZBH,144559,01,2001-08-07,2023-12-29,-0.074943


In [11]:
cnt = sp500ccm.groupby(['date'])['permno'].count().reset_index().rename(columns={'permno':'npermno'})
cnt.sample(4)

,date,npermno
142,2011-11-30,501
28,2002-05-31,500
185,2015-06-30,500
212,2017-09-29,501


In [12]:
sp500ccm.sample(10)

,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret
11663,2001-05-31,12319,HOMESTAKE MINING CO,43761410,11,1,1041,HM,005686,01,1957-03-01,2001-12-14,0.038523
31415,2003-11-28,22779,MOTOROLA INC,62007610,11,1,3663,MOT,007585,01,1957-03-01,2023-12-29,0.037694
93342,2011-12-30,59248,MOLSON COORS BREWING CO,60871R20,11,1,2082,TAP,003505,01,1976-07-01,2023-12-29,0.072678
138175,2017-12-29,19350,DEERE & CO,24419910,11,1,3523,DE,003835,01,1957-03-01,2023-12-29,0.048378
61795,2007-10-31,83862,E TRADE FINANCIAL CORP,26924610,11,3,6211,ETFC,063501,01,2004-04-01,2020-10-06,-0.146360
46167,2005-10-31,23887,CITIZENS COMMUNICATIONS CO,17453B10,11,1,4931,CZN,135484,01,2001-02-27,2017-03-17,-0.097417
169841,2022-04-29,75034,BAKER HUGHES CO,05722G10,11,3,3533,BKR,032106,01,1987-04-09,2023-12-29,-0.148036
137424,2017-11-30,23393,CHURCH & DWIGHT INC,17134010,11,1,2841,CHD,003026,01,2015-12-29,2023-12-29,0.046713
56522,2007-02-28,91650,SPECTRA ENERGY CORP,84756010,11,1,4923,SE,176282,01,2007-01-03,2017-02-27,-0.006508
6720,2000-10-31,41080,CUMMINS ENGINE INC,23102110,11,1,3519,CUM,003650,01,1965-01-07,2023-12-29,0.135699


In [16]:
sp500ccm.to_csv('sp500_constituents.csv', index=False)